In [1]:
import pandas as pd

In [2]:
#download link: https://www.kaggle.com/c/titanic/data
titanic_data=pd.read_csv('titanic_train.csv', quotechar='"')
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], 'columns', 
                 inplace=True)
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'].astype(str))
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [5]:
titanic_data = pd.get_dummies(titanic_data, columns=['Embarked'])
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [6]:
#find any missing data
titanic_data[titanic_data.isnull().any(axis=1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
5,0,3,1,NaN,0,0,8.4583,0,1,0
17,1,2,1,NaN,0,0,13.0000,0,0,1
19,1,3,0,NaN,0,0,7.2250,1,0,0
26,0,3,1,NaN,0,0,7.2250,1,0,0
28,1,3,0,NaN,0,0,7.8792,0,1,0
...,...,...,...,...,...,...,...,...,...,...
859,0,3,1,NaN,0,0,7.2292,1,0,0
863,0,3,0,NaN,8,2,69.5500,0,0,1
868,0,3,1,NaN,0,0,9.5000,0,0,1
878,0,3,1,NaN,0,0,7.8958,0,0,1


In [7]:
#delete or clear missing data
titanic_data = titanic_data.dropna()

In [29]:
from sklearn.cluster import MeanShift

analyzer = MeanShift(bandwidth=30)
analyzer.fit(titanic_data)

MeanShift(bandwidth=30, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [30]:
#this method helps to find a good bandwidth for your data
from sklearn.cluster import estimate_bandwidth
estimate_bandwidth(titanic_data)

30.446971266796215

In [32]:
#gives numeric label to grouped data
labels = analyzer.labels_

In [33]:
import numpy as np

np.unique(labels)

array([0, 1, 2, 3, 4], dtype=int64)

In [34]:
#add a new column 'cluster_group' to data
import numpy as np

titanic_data['cluster_group']= np.nan
data_length = len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i, 
            titanic_data.columns.get_loc('cluster_group')]=labels[i]

In [35]:
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
0,0,3,1,22.0,1,0,7.2500,0,0,1,0.0
1,1,1,0,38.0,1,0,71.2833,1,0,0,1.0
2,1,3,0,26.0,0,0,7.9250,0,0,1,0.0
3,1,1,0,35.0,1,0,53.1000,0,0,1,1.0
4,0,3,1,35.0,0,0,8.0500,0,0,1,0.0


In [36]:
titanic_data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,0.406162,2.236695,0.634454,29.699118,0.512605,0.431373,34.694514,0.182073,0.039216,0.775910,0.315126
std,0.491460,0.838250,0.481921,14.526497,0.929783,0.853289,52.918930,0.386175,0.194244,0.417274,0.690647
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,20.125000,0.000000,0.000000,8.050000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,2.000000,1.000000,28.000000,0.000000,0.000000,15.741700,0.000000,0.000000,1.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,1.000000,33.375000,0.000000,0.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,1.000000,1.000000,1.000000,4.000000


In [37]:
titanic_cluster_data = titanic_data.groupby(['cluster_group']).mean()
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
cluster_group,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000


In [25]:
#add a column 'Counts' to data frame to set to size of cluster

titanic_cluster_data['Counts'] = pd.Series(titanic_data.groupby(['cluster_group']).size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.388235,2.298529,0.651471,29.559191,0.505882,0.404412,25.415625,0.167647,0.041176,0.788235,680
1.0,0.741935,1.000000,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.000000,0.580645,31
2.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


In [26]:
titanic_data[titanic_data['cluster_group'] == 1].describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
count,31.000000,31.0,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.0,31.000000,31.0
mean,0.741935,1.0,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.0,0.580645,1.0
std,0.444803,0.0,0.444803,15.327994,1.006431,1.016001,50.203716,0.501610,0.0,0.501610,0.0
min,0.000000,1.0,0.000000,0.920000,0.000000,0.000000,133.650000,0.000000,0.0,0.000000,1.0
25%,0.500000,1.0,0.000000,22.000000,0.000000,0.000000,151.550000,0.000000,0.0,0.000000,1.0
50%,1.000000,1.0,0.000000,31.000000,0.000000,1.000000,164.866700,0.000000,0.0,1.000000,1.0
75%,1.000000,1.0,0.500000,41.500000,1.000000,2.000000,237.522900,1.000000,0.0,1.000000,1.0
max,1.000000,1.0,1.000000,64.000000,3.000000,4.000000,263.000000,1.000000,0.0,1.000000,1.0


In [38]:
titanic_data[titanic_data['cluster_group'] == 1]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
1,1,1,0,38.0,1,0,71.2833,1,0,0,1.0
3,1,1,0,35.0,1,0,53.1000,0,0,1,1.0
6,0,1,1,54.0,0,0,51.8625,0,0,1,1.0
34,0,1,1,28.0,1,0,82.1708,1,0,0,1.0
35,0,1,1,42.0,1,0,52.0000,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
835,1,1,0,39.0,1,1,83.1583,1,0,0,1.0
838,1,3,1,32.0,0,0,56.4958,0,0,1,1.0
867,0,1,1,31.0,0,0,50.4958,0,0,1,1.0
871,1,1,0,47.0,1,1,52.5542,0,0,1,1.0
